# Extracting wake and filtering it out of data

Restarting from the downsampled xrarray, extraction of active (without SWR) and quiet wake (with SWR) periods and removing these periods from the recordings.

## Load EMG and packages

In [ ]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path

%matplotlib widget

# mmap activated to not store everything in memory
All = np.load('RawDataChannelExtractedDS_2.npy', mmap_mode= 'r')
EMG = All[:, 11]


## High pass filter

In [ ]:
# Filter parameter :
f_lowcut = 200.
f_hicut = 400.
N = 4
fs = 1000
nyq = 0.5 * fs
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction


# Filter creation :
b, a = signal.butter(N, Wn, 'band')
filt_EMG = signal.filtfilt(b, a, EMG)

# Plot
times = np.arange(0, EMG.size/fs, 1./fs)
timesmin = np.arange(0, EMG.size/fs/60, 1./fs/60)

fig, ax = plt.subplots()
ax.plot(timesmin, filt_EMG)


## Continuous Wavelet Transform and projection calculation

In [ ]:
# Parameter and computation of CWT
w = 4.
freq = np.linspace(200, 400, 50)
widths = w*fs / (2*freq*np.pi)
EMGcwt = signal.cwt(EMG, signal.morlet2, widths, w=w)

# Projection calculation
absEMGcwt = np.absolute(EMGcwt)
proj_EMGcwt = np.sum(absEMGcwt, axis = 0)/50
sdproj_EMGcwt = np.std(proj_EMGcwt)
sd3proj_EMGcwt = sdproj_EMGcwt*3.5
sd05proj_EMGcwt = sdproj_EMGcwt*0.5



### Display subset 

Not necessary cell to run

In [ ]:
# Defining subset
start = 00000
end = 400000

tt = times[start:end]
EMGt = EMG[start:end]
EMGcwtt = EMGcwt[:, start:end]
proj_EMGcwtt = proj_EMGcwt[start:end]

plt.axhline(sdproj_EMGcwt, color='r') # horizontal
plt.axhline(sd3proj_EMGcwt, color='g') # horizontal
plt.axhline(sd05proj_EMGcwt, color='b') # horizontal
plt.plot(tt, EMGt)
plt.plot(tt, proj_EMGcwtt)
plt.show()

## WARNING: Plot only short subsets (~ 10 s), too memory consuming otherwise
#plt.pcolormesh(tt, freq, np.abs(EMGcwt), cmap='viridis', shading='gouraud')
#plt.plot(tt, EMGt)
#plt.show()


In [ ]:
plt.close('all')


## Interpretating signal to extract wake (active and quiet) and sleep

In [ ]:
# Assigning values wake (1, 2) and sleep (0)
numpnts = EMG.size
EMGstatusRaw = np.zeros(numpnts)
for ind in range(numpnts):
    if proj_EMGcwt[ind]<sd05proj_EMGcwt:
        EMGstatusRaw[ind] = 0
    elif proj_EMGcwt[ind]>sd3proj_EMGcwt:
        EMGstatusRaw[ind] = 2
    else: 
        EMGstatusRaw[ind] = 1

# Expanding borders for wake (1, 2) and sleep (0) to ±1 s around detected muscular activity
EMGstatusRaw2 = np.zeros(numpnts)
for ind in range(numpnts):
    if EMGstatusRaw[ind]>1:
       EMGstatusRaw2[ind-1000:ind+1000] = 2
    elif EMGstatusRaw[ind]==1:
        for ind2 in range(ind-1000, ind+1000):
            if ind2==numpnts:
                break
            elif EMGstatusRaw2[ind2]<2:
                EMGstatusRaw2[ind2] = 1

In [ ]:
# if needed, plot of the two layers of filters
plt.close('all')
plt.plot(tt, EMGstatusRaw)
plt.plot(tt, EMGstatusRaw2)
plt.show()

## To Create boolean masks and to save recordings without movement
#### Creating two masks: one conservative excludes weak muscular activity that can be quiet wake or dozing and one liberal that includes weak muscular activity

In [ ]:
EMGStatusBoolLib = (EMGstatusRaw2>1)
EMGStatusBoolCons = (EMGstatusRaw2>0)


#### Removing wake time from the recordings.
Two files created one with 0 instead of signal during wake and one with the wake time removed

In [ ]:
LFP = All[:, 1:4]
LFPwake0 = LFP.copy()
LFPwake0[EMGStatusBoolLib] = 0
np.save('LFPwake0.npy', LFPwake0)

LFPwakeremoved = LFP.copy()
LFPwakeremoved = LFPwakeremoved[~EMGStatusBoolLib, :]
np.save('LFPwakeremoved_2.npy', LFPwakeremoved)

data = {
    'EMGstatus': EMGstatusRaw2,
    'BooleanLiberal' : EMGStatusBoolLib,
    'BooleanConservative' : EMGStatusBoolCons
}
WakeFrame = pd.DataFrame(data, columns=['EMGstatus', 'BooleanLiberal', 'BooleanConservative'])
WakeFrame.to_pickle('EMGframeBoolean_2.pkl')
# if done and no intention to display for assessment
#%reset
#plt.close('all')

In [ ]:
WakeFrame

In [ ]:
# Display on a subset (not a necessary step)
start = 1000
end = 2000

LFP = All[start:end, 1:4]
EMGStatusBoolLibt = EMGStatusBoolLib[start:end]
tt2 = tt[start:end]
EMGstatusRaw3 = EMGstatusRaw2[start:end]*2000
tentative = LFP.copy()

# to display LFP with wake time kept but signal set at 0
#tentative[EMGStatusBoolLibt] = 0
#plt.close('all')
#plt.plot(tt2, tentative)
#plt.plot(tt2, EMGstatusRaw3)
#plt.show()

# to display LFP with wake time removed.
tentative = tentative[~EMGStatusBoolLibt, :]
print(tentative.shape)
tt3 = tt[start:tentative.shape[0]]
plt.close('all')
plt.plot(tt3, tentative)
plt.show()
